[View in Colaboratory](https://colab.research.google.com/github/kyokicchi/ggl_an_codes/blob/master/trainLGBmodel.ipynb)

In [1]:
!pip install lightgbm

    100% |████████████████████████████████| 1.1MB 7.7MB/s 


In [0]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from google.colab import drive
from google.colab import files
from sklearn.model_selection import train_test_split
import pickle

In [3]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
%cd ../content/gdrive/My Drive/projects/ggl_an/
%ls

/content/gdrive/My Drive/projects/ggl_an
model_LGB.sav  model_XGB.sav  user_test.csv    user_train.csv
model_NN.h5    output.csv     user_test_n.csv  user_train_n.csv


In [0]:
s_data = 'user_train.csv'
filename = 'model_LGB.sav'
s_col_tgt = 'transactionRevenue'
l_col_drop = [s_col_tgt]

In [6]:
%%time
df_train = pd.read_csv(s_data, index_col = 0)

CPU times: user 2.94 s, sys: 519 ms, total: 3.46 s
Wall time: 5.02 s


<string>:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.


In [9]:
%%time
df_y = df_train[s_col_tgt]
df_x = df_train.drop(l_col_drop, axis=1)

CPU times: user 43.4 ms, sys: 107 ms, total: 150 ms
Wall time: 149 ms


In [10]:
%%time
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y)
x_eval, x_valid, y_eval, y_valid = train_test_split(x_test, y_test)



CPU times: user 979 ms, sys: 47.4 ms, total: 1.03 s
Wall time: 1.03 s


In [0]:

params = {
    'learning_rate': 0.08,
    'n_estimators': 1000,
    'num_leaves': 128,
    'subsample': 0.2217,
    'colsample_bytree': 0.6810,
    'min_split_gain': np.power(10.0, -4.9380),
    'reg_alpha': np.power(10.0, -3.2454),
    'reg_lambda': np.power(10.0, -4.8571),
    'min_child_weight': np.power(10.0, 2),
    'silent': True
}

model = lgb.LGBMRegressor(**params)



In [17]:
%%time
model.fit(x_train, 
          y_train,
          eval_set=[(x_train, y_train),(x_eval, y_eval)],
          eval_metric='rmse',
          early_stopping_rounds = 300)

[1]	training's l2: 8.18662	training's rmse: 2.86123	valid_1's l2: 7.73974	valid_1's rmse: 2.78204
Training until validation scores don't improve for 300 rounds.
[2]	training's l2: 7.83252	training's rmse: 2.79866	valid_1's l2: 7.39949	valid_1's rmse: 2.7202
[3]	training's l2: 7.53654	training's rmse: 2.74528	valid_1's l2: 7.11135	valid_1's rmse: 2.66671
[4]	training's l2: 7.28971	training's rmse: 2.69995	valid_1's l2: 6.8709	valid_1's rmse: 2.62124
[5]	training's l2: 7.06006	training's rmse: 2.65708	valid_1's l2: 6.65733	valid_1's rmse: 2.58018
[6]	training's l2: 6.87297	training's rmse: 2.62164	valid_1's l2: 6.48487	valid_1's rmse: 2.54654
[7]	training's l2: 6.70875	training's rmse: 2.59013	valid_1's l2: 6.3211	valid_1's rmse: 2.51418
[8]	training's l2: 6.55133	training's rmse: 2.55956	valid_1's l2: 6.18107	valid_1's rmse: 2.48618
[9]	training's l2: 6.4267	training's rmse: 2.53509	valid_1's l2: 6.07134	valid_1's rmse: 2.46401
[10]	training's l2: 6.3115	training's rmse: 2.51227	valid_1

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.681,
       importance_type='split', learning_rate=0.08, max_depth=-1,
       min_child_samples=20, min_child_weight=100.0,
       min_split_gain=1.153453257821093e-05, n_estimators=1000, n_jobs=-1,
       num_leaves=128, objective=None, random_state=None,
       reg_alpha=0.0005683292391383383, reg_lambda=1.3896326196366016e-05,
       silent=True, subsample=0.2217, subsample_for_bin=200000,
       subsample_freq=0)

In [18]:
%%time
pickle.dump(model, open(filename, 'wb'))

# files.download(filename) 

CPU times: user 115 ms, sys: 1.01 ms, total: 116 ms
Wall time: 696 ms
